In [1]:
from tkinter import *
import random

GAME_WIDTH = 700
GAME_HEIGHT = 700
SPEED = 100
SPACE_SIZE = 50
BODY_PARTS = 3
SNAKE_COLOR = "#00FF00"
FOOD_COLOR = "#FF0000"
BACKGROUND_COLOR = "#000000"

class Snake:
    def __init__(self):
        self.body = [[100, 50], [50, 50], [0, 50]]  # Initial snake body (3 parts)
        self.direction = "Right"
        self.color = SNAKE_COLOR
        self.score = 0
    
    def move(self):
        head = self.body[0]
        new_head = []

        if self.direction == "Up":
            new_head = [head[0], head[1] - SPACE_SIZE]
        elif self.direction == "Down":
            new_head = [head[0], head[1] + SPACE_SIZE]
        elif self.direction == "Left":
            new_head = [head[0] - SPACE_SIZE, head[1]]
        elif self.direction == "Right":
            new_head = [head[0] + SPACE_SIZE, head[1]]
        
        self.body = [new_head] + self.body[:-1]  # Move the snake by adding the new head and removing the last body part
    
    def grow(self):
        tail = self.body[-1]
        self.body.append(tail)  # Add a new body part to the end
    
    def draw(self, canvas):
        canvas.delete("snake")  # Clear previous snake
        for part in self.body:
            canvas.create_rectangle(part[0], part[1], part[0] + SPACE_SIZE, part[1] + SPACE_SIZE, fill=self.color, tag="snake")

class Food:
    def __init__(self):
        self.coordinates = None
        self.food_item = None
    
    def spawn_food(self, canvas):
        # Correcting the float-to-integer conversion issue
        x = random.randint(0, int(GAME_WIDTH / SPACE_SIZE) - 1) * SPACE_SIZE
        y = random.randint(0, int(GAME_HEIGHT / SPACE_SIZE) - 1) * SPACE_SIZE
        self.coordinates = [x, y]
        
        # Remove any previous food before drawing a new one
        if self.food_item:
            canvas.delete(self.food_item)  # Delete the previous food from the canvas

        # Draw the new food item on the canvas
        self.food_item = canvas.create_oval(self.coordinates[0], self.coordinates[1], 
                                            self.coordinates[0] + SPACE_SIZE, self.coordinates[1] + SPACE_SIZE, 
                                            fill=FOOD_COLOR, tag="food")

def next_turn(snake, food, canvas, label):
    snake.move()
    if check_collisions(snake, food, canvas):
        snake.grow()
        food.spawn_food(canvas)  # Spawn a new food item after eating
        snake.score += 1
        label.config(text="Score:{}".format(snake.score))
    
    snake.draw(canvas)
    window.after(SPEED, next_turn, snake, food, canvas, label)  # Keep the game running at the specified speed

def change_direction(new_direction, snake):
    if (new_direction == "Up" and snake.direction != "Down") or \
       (new_direction == "Down" and snake.direction != "Up") or \
       (new_direction == "Left" and snake.direction != "Right") or \
       (new_direction == "Right" and snake.direction != "Left"):
        snake.direction = new_direction

def check_collisions(snake, food, canvas):
    head = snake.body[0]
    
    # Check for collision with walls
    if head[0] < 0 or head[0] >= GAME_WIDTH or head[1] < 0 or head[1] >= GAME_HEIGHT:
        game_over(snake, canvas)
        return False
    
    # Check for collision with itself
    if head in snake.body[1:]:
        game_over(snake, canvas)
        return False
    
    # Check for collision with food
    if head == food.coordinates:
        return True  # The snake eats the food
    
    return False

def game_over(snake, canvas):
    canvas.delete("all")  # Clear the canvas
    canvas.create_text(GAME_WIDTH / 2, GAME_HEIGHT / 2, text="Game Over", fill="white", font=("consoles", 40))
    canvas.create_text(GAME_WIDTH / 2, GAME_HEIGHT / 2 + 50, text="Final Score: {}".format(snake.score), fill="white", font=("consoles", 30))

window = Tk()
window.title("Snake Game")
window.resizable(False, False)

score = 0
label = Label(window, text="Score:{}".format(score), font=("consoles", 40))
label.pack()

canvas = Canvas(window, bg=BACKGROUND_COLOR, height=GAME_HEIGHT, width=GAME_WIDTH)
canvas.pack()

window.update()

window_width = window.winfo_width()
window_height = window.winfo_height()
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()

x = int((screen_width / 2) - (window_width / 2))
y = int((screen_height / 2) - (window_height / 2))

window.geometry(f"{window_width}x{window_height}+{x}+{y}")

snake = Snake()
food = Food()

food.spawn_food(canvas)
snake.draw(canvas)

# Keybindings for controlling the snake
window.bind("<Up>", lambda event: change_direction("Up", snake))
window.bind("<Down>", lambda event: change_direction("Down", snake))
window.bind("<Left>", lambda event: change_direction("Left", snake))
window.bind("<Right>", lambda event: change_direction("Right", snake))

# Start the game loop
next_turn(snake, food, canvas, label)

window.mainloop()